In [1]:
!nvidia-smi

Sat Dec  4 15:03:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.118.02   Driver Version: 440.118.02   CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P2000        Off  | 00000000:65:00.0  On |                  N/A |
| 56%   54C    P0    19W /  75W |    806MiB /  5044MiB |     14%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install  transformers

     |████████████████████████████████| 3.1 MB 655 kB/s eta 0:00:01
  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
     |████████████████████████████████| 701 kB 50.7 MB/s eta 0:00:01
  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
     |████████████████████████████████| 61 kB 77 kB/s s eta 0:00:01
     |████████████████████████████████| 61 kB 715 kB/s  eta 0:00:01
     |████████████████████████████████| 138 kB 61.5 MB/s eta 0:00:01


In [3]:
!pip install pytorch-lightning==1.2.7

  Using cached pytorch_lightning-1.2.7-py3-none-any.whl (830 kB)
     |████████████████████████████████| 5.8 MB 639 kB/s eta 0:00:01
     |████████████████████████████████| 132 kB 42.2 MB/s eta 0:00:01
     |████████████████████████████████| 329 kB 38.0 MB/s eta 0:00:01
  Using cached torch-1.10.0-cp38-cp38-manylinux1_x86_64.whl (881.9 MB)
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached tensorboard_plugin_wit-1.8.0-py3-none-any.whl (781 kB)
     |████████████████████████████████| 97 kB 767 kB/s  eta 0:00:01
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
     |████████████████████████████████| 126 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 77.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 69.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 43.3 MB/s eta 0:00:01
     |████████████████████████████████| 

In [6]:
import pandas as pd
import numpy as np
import torch

In [7]:
from pathlib import Path

In [8]:
from torch.utils.data import Dataset,DataLoader

In [9]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [10]:
from sklearn.model_selection import train_test_split

import textwrap

In [11]:
from transformers import T5TokenizerFast

In [12]:
from transformers import T5ForConditionalGeneration

In [13]:
from tqdm.auto import tqdm

In [14]:
from transformers import AdamW

In [19]:
!pip install seaborn
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
sns.set(style='whitegrid', palette='muted',font_scale=1.2)
rcParams['figure.figsize']=16,10

     |████████████████████████████████| 292 kB 624 kB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 1.3 MB/s eta 0:00:011     |██████████████████████▌         | 2.2 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 51.8 MB/s eta 0:00:01
     |████████████████████████████████| 884 kB 38.7 MB/s eta 0:00:01


In [20]:
pl.seed_everything(42)

Global seed set to 42


42

In [21]:
#df=pd.read_csv('train_xsum.csv')

In [22]:
#train_df=df[['document','summary']]

In [23]:
#tf=pd.read_csv('val_xsum.csv')

In [24]:
#test_df=tf[['document','summary']]

In [25]:
#train_df,test_df=train_test_split(df,test_size=0.1)
#train_df.shape, test_df.shape

In [26]:
class NewsSummaryDataset(Dataset):
    def __init__(
    self,
    data: pd.DataFrame,
    tokenizer: T5TokenizerFast,
    text_max_token_len: int=512,
    summary_max_token_len: int=128
    ):
        self.tokenizer= tokenizer
        self.data=data
        self.text_max_token_len=text_max_token_len
        self.summary_max_token_len=summary_max_token_len
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index: int):
        data_row=self.data.iloc[index]
        
        text=data_row['document']
        
        text_encoding=tokenizer(
        text,
        max_length=self.text_max_token_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt')
        
        summary_encoding=tokenizer(
        data_row['summary'],
        max_length=self.summary_max_token_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt')
        
        labels=summary_encoding['input_ids']
        labels[labels==0]= -100
                 
        return dict(
        text=text,
        summary=data_row['summary'],
        text_input_ids=text_encoding['input_ids'].flatten(),
        text_attention_masks=text_encoding['attention_mask'].flatten(),
        labels=labels.flatten(),
        labels_attention_mask=summary_encoding['attention_mask'].flatten()
        )
                 
        

In [27]:
class NewsSummaryDataModule(pl.LightningDataModule):
    def __init__(
    self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5TokenizerFast,
        batch_size:int=8,
        text_max_token_len:int=512,
        summary_max_token_len:int=128
    ):
        super().__init__()
        
        self.train_df=train_df
        self.test_df=test_df
        self.batch_size=batch_size
        self.tokenizer=tokenizer
        self.text_max_token_len=text_max_token_len
        self.summary_max_token_len=summary_max_token_len
        
    def setup(self,stage=None):
        self.train_dataset=NewsSummaryDataset(
        self.train_df,
        self.tokenizer,
        self.text_max_token_len,
        self.summary_max_token_len
        )
        
        self.test_dataset=NewsSummaryDataset(
        self.test_df,
        self.tokenizer,
        self.text_max_token_len,
        self.summary_max_token_len
        )
        
    def train_dataloader(self):
        return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=2)
        
    def val_dataloader(self):
        return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        shuffle=False,
        num_workers=2)
    
    def val_dataloader(self):
        return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        shuffle=False,
        num_workers=2)
        

In [28]:
model_name='t5-base'

In [31]:
!pip install ipywidgets
from ipywidgets import IntProgress

     |████████████████████████████████| 121 kB 660 kB/s eta 0:00:01
     |████████████████████████████████| 243 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 6.0 MB/s eta 0:00:01


In [32]:
tokenizer=T5TokenizerFast.from_pretrained(model_name)

In [33]:
#train_df=train_df.dropna()

In [34]:
#train_df.shape

In [35]:
#test_df=test_df.dropna()
#test_df.shape

In [38]:
# text_token_counts, summary_token_counts=[],[]

# for _,row in train_df.iterrows():
#     text_token_count=len(tokenizer.encode(row['document']))
#     text_token_counts.append(text_token_count)

#     summary_token_count=len(tokenizer.encode(row['summary']))
#     summary_token_counts.append(summary_token_count)
    

NameError: name 'train_df' is not defined

In [ ]:
#fig,(ax1,ax2)=plt.subplots(1,2)

#sns.histplot(text_token_counts, ax=ax1)
#ax1.set_title('full text token counts')

#sns.histplot(summary_token_counts, ax=ax2)
#ax2.set_title('full summary token counts')

In [ ]:
#n_epochs=1
#batch_size=8
#data_module=NewsSummaryDataModule(train_df,test_df,tokenizer,batch_size=batch_size)

# MODEL

In [42]:
class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model=T5ForConditionalGeneration.from_pretrained(model_name, return_dict=True)
        
    def forward(self,input_ids, attention_mask, decoder_attention_mask, labels=None):
        output=self.model(
        input_ids,
        attention_mask=attention_mask,
        labels=labels,
        decoder_attention_mask=decoder_attention_mask)
        
        return output.loss, output.logits
    
    def training_step(self, batch, batch_idx):
        input_ids=batch['text_input_ids']
        attention_mask=batch['text_attention_masks']
        labels=batch['labels']
        labels_attention_mask=batch['labels_attention_mask']
        
        loss, outputs=self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels)
        
        self.log('train_loss',loss,prog_bar=True,logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_ids=batch['text_input_ids']
        attention_mask=batch['text_attention_masks']
        labels=batch['labels']
        labels_attention_mask=batch['labels_attention_mask']
        
        loss, outputs=self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels)
        
        self.log('val_loss',loss,prog_bar=True,logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids=batch['text_input_ids']
        attention_mask=batch['text_attention_masks']
        labels=batch['labels']
        labels_attention_mask=batch['labels_attention_mask']
        
        loss, outputs=self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels)
        
        self.log('test_loss',loss,prog_bar=True,logger=True)
        return loss
    
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.001)

In [17]:
#model=NewsSummaryModel()

NameError: name 'T5ForConditionalGeneration' is not defined

In [62]:
#checkpoint_callback=ModelCheckpoint(
#dirpath='checkpoints',
#filename='best-checkpoint',
#save_top_k=1,
#verbose=True,
#monitor='val_loss',
#mode='min')

#logger=TensorBoardLogger('lightning_logs', name='news-summary')

#trainer= pl.Trainer(
#logger=logger,
#checkpoint_callback=checkpoint_callback,
#max_epochs=n_epochs,

#progress_bar_refresh_rate=30
#)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/tawmo/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)


In [63]:
#trainer.fit(model, data_module)


  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 25502: val_loss reached 2.11609 (best 2.11609), saving model to "/home/tawmo/checkpoints/best-checkpoint.ckpt" as top 1


1

In [64]:
#trained_model=NewsSummaryModel.load_from_checkpoint(
#trainer.checkpoint_callback.best_model_path)
#trained_model.freeze()

In [39]:
import pickle
#filename='T5_Xsum'
#pickle.dump(model,open(filename,'wb'))

In [43]:
T5_CNN=pickle.load(open('T5_CNN','rb'))

In [ ]:
#torch.save(trained_model,'CNN_trained_t5')

In [ ]:
#modelt5=torch.load('CNN_trained_t5')

In [44]:
def summarize(text):
    text_encoding=tokenizer(
    text,
    max_length=512,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors='pt')
    
    generated_ids=T5_CNN.model.generate(
    input_ids=text_encoding['input_ids'],
    attention_mask=text_encoding['attention_mask'],
    max_length=150,
    num_beams=2,
    repetition_penalty=2.5,
    length_penalty=1.0,
    early_stopping=True)
    
    preds=[
        tokenizer.decode(gen_id, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        for gen_id in generated_ids
    ]
    
    return "".join(preds)

In [45]:
test_data=pd.read_csv("test_xsum.csv")
print(test_data)

FileNotFoundError: [Errno 2] No such file or directory: 'test_xsum.csv'

In [24]:
test_data=test_data.dropna()
test_data.shape

(11333, 4)

In [25]:
test_datax=test_data['document']
dict1={}
for i in test_datax:
    summary=summarize(i)
    dict1[i]=summary

In [47]:
text = 'At least 31 people were killed and eight others injured after terrorists attack on a bus in Mali, a source told news agency Sputnik. The incident took place not far from the eastern Malian town of Bandiagara, the Mopti region, according to the source.Malis situation was destabilized in 2012 when the Tuareg militants seized vast territories in the northern part of the country. The conflict escalated even more over the activities of Islamists, forces loyal to former Libyan leader Muammar Gaddafi as well as French interference.'
summary = summarize(text)
print(summary)

The incident took place not far from the eastern Malian town of Bandiagara. The conflict escalated even more over the activities of Islamists, forces loyal to Muammar Gaddafi.


In [41]:
len(dict1)

11329

In [42]:
test_datay = pd.DataFrame(list(dict1.items()),columns = ['document','summary'])

In [43]:
len(test_datay)

11329

In [44]:
test_datay

,document,summary
0,"Fast forward about 20 years, and it's fair to ...",Frasers Hospitality is one of the world's bigg...
1,"""The accident meant the motorway was closed, m...",Northern Ireland have three points from their ...
2,The Sunday Times says the missile veered off c...,"SNP's Nicola Sturgeon called for ""full disclos..."
3,A spokesman for Palm Beach Gardens police in F...,"Jerome Barson died two weeks after the crash, ..."
4,The International Tribunal for the Law of the ...,Ivorian tribunal for the Law of the Sea rules ...
...,...,...
11324,Messages posted on social media claimed the us...,Messages posted on social media claimed the us...
11325,Media playback is unsupported on your device\n...,Cancer Research UK has organised the Slide on ...
11326,Chris Erskine crossed low for Kris Doolan to t...,Adam Barton and Steven Lawless came close to d...
11327,The main feature was a drop in bank shares.\nT...,"RBS shares fell by 4.4%, Barclays by 2.8% and ..."


In [45]:
test=test_datay['summary']

In [46]:
highlights=test_data['summary']

In [47]:
test.to_csv('predictions.csv')

In [48]:
highlights.to_csv('originals.csv')

In [87]:
test=pd.read_csv('predictions.csv')

In [89]:
test

0        Frasers Hospitality is one of the world's bigg...
1        Northern Ireland have three points from their ...
2        SNP's Nicola Sturgeon called for "full disclos...
3        Jerome Barson died two weeks after the crash, ...
4        Ivorian tribunal for the Law of the Sea rules ...
                               ...                        
11328    Messages posted on social media claimed the us...
11329    Cancer Research UK has organised the Slide on ...
11330    Adam Barton and Steven Lawless came close to d...
11331    RBS shares fell by 4.4%, Barclays by 2.8% and ...
11332    The birds set up nests in the banks of Low Gel...
Name: summary, Length: 11333, dtype: object

In [88]:
test=test['summary']

In [36]:
highlights=highlights['summary']

In [90]:
len(test)

11333

In [52]:
len(highlights)

11333

In [53]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

In [91]:
rouge_output = rouge.get_scores(test, highlights,avg=True)

In [92]:
print(rouge_output)

{'rouge-1': {'r': 0.23908082368441946, 'p': 0.14428708763999087, 'f': 0.17607629799534774}, 'rouge-2': {'r': 0.033661559208618755, 'p': 0.01892540286668346, 'f': 0.023595024684933782}, 'rouge-l': {'r': 0.20321049406804223, 'p': 0.1222189678454218, 'f': 0.14929924729865926}}


In [78]:
from nltk.translate.bleu_score import corpus_bleu

In [93]:
score=corpus_bleu(highlights,test)

In [94]:
score 

1.0461309381300793e-231